<a href="https://colab.research.google.com/github/khaled-kk/Image-Processing/blob/main/IPM1S23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.io import imread
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
import argparse

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
# Load the Inception V3 model
model = InceptionV3(weights='imagenet', include_top=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def predict_images(model_name, image_dir,QF):
    # Load the model
    if model_name == 'InceptionV3':
        model = InceptionV3(weights='imagenet',include_top=True)
        img_width, img_height = 299, 299
    else:
        raise ValueError('Invalid model name')

    # Set up the data generator
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    # Get the list of image filenames
    image_filenames = sorted(os.listdir(image_dir))
    #image_filenames = sorted((image_dir))
    # Create a generator for the images in batches
    def image_batch_generator(filenames, batch_size):
        for i in range(0, len(filenames), batch_size):
            batch_filenames = filenames[i:i+batch_size]
            batch_images = []
            for filename in batch_filenames:
                image_path = os.path.join(image_dir, filename)
                print(image_path)
                img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
                img = np.array(img)
                img = preprocess_input(img)
                batch_images.append(img)
            yield np.array(batch_images)



    # Predict the classes of the images in batches
    predictions = []
    for batch_images in image_batch_generator(image_filenames, batch_size=32):
        batch_predictions = model.predict(batch_images)
        predictions.append(batch_predictions)
    predictions = np.concatenate(predictions)
    decoded_preds = decode_predictions(predictions, top=5)
    #print(decoded_preds[0][0][0])

    predsTop5Index=[]
    predsTop1Index=[]
    for i in range(len(decoded_preds)):
        top5 = []
        class_index, class_name, confidence = decoded_preds[i][0]
        predsTop1Index.append((class_index, class_name))
        for j in range(5):
            class_index, class_name, confidence = decoded_preds[i][j]
            top5.append((class_index,class_name))
        predsTop5Index.append(top5)

    with open(f"Prediction{model_name}-{QF}.txt", 'w') as f:
        for i in range(len(predsTop1Index)):
            predicted_class,predicted_label =predsTop1Index[i]
            f.write(f'{predicted_class} {predicted_label} \n')

    with open(f"PredictionTOP5{model_name}-{QF}.txt", 'w') as f:
        for i in range(len(predsTop5Index)):
            f.write(f'{predsTop5Index[i]} \n')

In [ ]:
# specify the path to the main folder
main_folder ='/content/drive/MyDrive/Sorted QF folders/'
#'/content/drive/MyDrive/Sorted QF folders/'
# loop through all the files in the main folder and its subfolders
for root, dirs, files in os.walk(main_folder):
    for file in dirs:
        # do something with the file
        print(f"Processing file: {file}")
        path = os.path.join(root, file)
        predict_images('InceptionV3',path,file)

In [ ]:
def get_label_mapping(mapping_file):
    mapping = {}
    with open(mapping_file) as f:
        for line in f:
            line = line.strip()
            if line:
                id_, label = line.split()
                mapping[id_] = label
    return mapping

def get_ground_truth(ground_truth_file):
    with open(ground_truth_file) as f:
        ground_truth = [int(line.strip()) for line in f]
    return ground_truth

def get_labels(labels_file):
    labels = {}
    with open(labels_file) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split(None, 1)
                id_ = parts[0]
                label = parts[1] if len(parts) > 1 else ''
                labels[id_] = label
    return labels



In [ ]:
mapping_labels = get_label_mapping('ILSVRC2012_mapping.txt')

In [ ]:
GT = get_ground_truth('ILSVRC2012_validation_ground_truth.txt')

In [ ]:
mapperGT = []
for item in GT:
    mapperGT.append(mapping_labels[str(item)])

In [ ]:
def get_predicted_labels(labels_file):
    labels = []
    with open(labels_file) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split(None, 2)
                labels.append(parts[0])
    return labels

In [ ]:
def get_predicted_Top5_labels(labels_file):
    labels = []
    with open(labels_file) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split(None)
                arr = [parts[0],parts[2],parts[4],parts[6],parts[8]]
                outarr = [s.replace("[('", '').replace("',", '').replace("('", '') for s in arr]
                labels.append(outarr)

    return labels

In [ ]:
def truePrediction(predicted_labels,mapper):
    hits= 0
    idx = 0
    top1 = 0
    for item in predicted_labels:
        if item == mapper[idx]:
            hits= hits+1
        idx=idx+1
    return hits

In [ ]:
def findCount(QFlabels,model):
    QFcount =[]
    idx=0
    for file in QFlabels:
        filepath = 'Prediction'+model+'-'+file+'.txt'
        predicted= get_predicted_labels(filepath)
        count = truePrediction(predicted,mapperGT)
        QFcount.append(count)
    return QFcount

In [ ]:
QFlabels=['QF-10','QF-20','QF-30','QF-40','QF-50','QF-60','QF-70','QF-80','QF-90','QF-100','Original']

In [ ]:
InceptionV3QFcounts =findCount(QFlabels,'InceptionV3')
InceptionV3QFcounts

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=90)
plt.bar(QFlabels, InceptionV3QFcounts)
plt.xlabel('QF Selector')
plt.ylabel('Image Count')
plt.title('Bar graph of Quality Factor counts of InceptionV3')
plt.show()

In [ ]:
c1 =[]

for i in range(len(InceptionV3QFcounts)):
    c1.append((InceptionV3QFcounts[i]/2000)*100)


plt.figure(figsize=(10,8))
# Create the plot
plt.plot(QFlabels, c1)

# Add labels and title
plt.xlabel('Quality Factor (QF)')
plt.ylabel('Top1 Accuracy (%)')
plt.title('Top-1 Accuracies for Original and Compressed Images')

# Show the plot
plt.show()

In [ ]:
def Top5(QFlabels,mapper,model):
    Top5count =[0]*len(QFlabels)
    idx = 0
    for file in QFlabels:
        filepath = 'PredictionTOP5'+model+'-'+file+'.txt'
        predicted_labels = get_predicted_Top5_labels(filepath)
        for i in range(len(predicted_labels)):
            for j in range(len(predicted_labels[i])):
                if(mapper[i]==predicted_labels[i][j]):
                    Top5count[idx]=Top5count[idx]+1
                    break
        idx=idx+1
    return Top5count

In [ ]:
Top5InceptionV3 = Top5(QFlabels,mapperGT,'InceptionV3')

c5 =[]
for i in range(len(Top5InceptionV3)):
    c5.append((Top5InceptionV3[i]/2000)*100)


plt.figure(figsize=(10,8))
# Create the plot
plt.plot(QFlabels, c5)
plt.legend('InceptionV3')
# Add labels and title
plt.xlabel('Quality Factor (QF)')
plt.ylabel('Top5 Accuracy (%)')
plt.title('Top-5 Accuracies for Original and Compressed Images')
# Show the plot
plt.show()

In [ ]:
def HRS_QF(QFlabels,mapper,model):
    HRScount =[0]*len(QFlabels)

    for idx in range(0,1000):
        labelIdx=0
        for file in QFlabels:
            filepath = 'Prediction'+model+'-'+file+'.txt'
            predicted_label= get_predicted_labels(filepath)
            print(idx)
            if predicted_label[idx] == mapper[idx]:
                HRScount[labelIdx]= HRScount[labelIdx]+1
                break
            labelIdx=labelIdx+1
    return HRScount

In [ ]:
HRSInceptionV3QF = HRS_QF(QFlabels,mapperGT,'InceptionV3')

In [ ]:
y = HRSInceptionV3QF
plt.figure(figsize=(10,8))
plt.bar(QFlabels, y)
plt.xlabel('Quality Factors')
plt.ylabel('Image Count')
plt.title('Bar graph of Quality Factor counts for InceptionV3')
plt.show()

In [ ]:
def calculate_ssim(original_img, compressed_img):
    ssim = structural_similarity(original_img, compressed_img, multichannel=True)
    return ssim

In [ ]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

In [ ]:
def ssim(img1, img2):
    return calculate_ssim(img1, img2)

In [ ]:
def get_compression_ratio(input_file, output_file):
    input_size = os.path.getsize(input_file)
    output_size = os.path.getsize(output_file)
    return input_size / output_size

In [ ]:
def get_image_by_index(folder_path, index):
    # Get a list of all image files in the folder
    image_files = sorted(os.listdir(folder_path))

    # Make sure the index is within range
    if index < 0 or index >= len(image_files):
        raise ValueError("Invalid index")

    # Get the file path of the image at the given index
    image_path = os.path.join(folder_path, image_files[index])

    # Read the image file and return it
    return image_path


In [ ]:
  def psnrImage(main_folder,index,labels):
      originalpath = "/content/drive/MyDrive/Sorted QF folders/Original/"
      originalImgpath = get_image_by_index(originalpath,index)
      originalImg = cv2.imread(originalImgpath)

      images=[]
      psnrList =[]
      ssimList =[]
      # loop through all the files in the main folder and its subfolders
      for file in labels:
              path = os.path.join(main_folder, file)
              imgpath = get_image_by_index(path, index)
              img=cv2.imread(imgpath)
              images.append(img)

      for i in range(len(images)):
          img2_resized = cv2.resize(images[i], (originalImg.shape[1], originalImg.shape[0]))
          psnrvalue = psnr(originalImg,img2_resized)
          ssimvalue = ssim(originalImg,img2_resized)
          psnrList.append(psnrvalue)
          ssimList.append(ssimvalue)
      return psnrList,ssimList


In [ ]:
def CRImage(main_folder,index,labels):
    originalpath = "/content/drive/MyDrive/Sorted QF folders/Original/"
    originalImgpath = get_image_by_index(originalpath,index)

    images=[]
    CRList =[]
    # loop through all the files in the main folder and its subfolders
    for file in labels:
            path = os.path.join(main_folder, file)
            imgpath = get_image_by_index(path, index)
            images.append(imgpath)
    for i in range(len(images)):
        CRvalue = get_compression_ratio(images[i],originalpath)
        CRList.append(CRvalue)
    return CRList

In [ ]:
QF1labels=['QF-10','QF-20','QF-30','QF-40','QF-50','QF-60','QF-70','QF-80','QF-90','QF-100']
Img1PSNR,Img1ssim = psnrImage("/content/drive/MyDrive/Sorted QF folders/",0,QF1labe ls)
Img2PSNR,Img2ssim = psnrImage("/content/drive/MyDrive/Sorted QF folders/",1,QF1labels)
Img3PSNR,Img3ssim= psnrImage("/content/drive/MyDrive/Sorted QF folders/",2,QF1labels)
Img4PSNR,Img4ssim = psnrImage("/content/drive/MyDrive/Sorted QF folders/",3,QF1labels)
Img651PSNR,Img651ssim = psnrImage("/content/drive/MyDrive/Sorted QF folders/",651,QF1labels)


In [ ]:
Img1CR = CRImage("/content/drive/MyDrive/Sorted QF folders/",0,QF1labels)
Img2CR = CRImage("/content/drive/MyDrive/Sorted QF folders/",1,QF1labels)
Img3CR = CRImage("/content/drive/MyDrive/Sorted QF folders/",2,QF1labels)
Img4CR = CRImage("/content/drive/MyDrive/Sorted QF folders/",3,QF1labels)
Img651CR = CRImage("/content/drive/MyDrive/Sorted QF folders/",651,QF1labels)


In [ ]:
#Plot CR vs PSNR
plt.figure(figsize=(10,8))
# Create the plot
plt.plot(Img1CR, Img1PSNR, '-o')
plt.plot(Img2CR, Img2PSNR, '-o')
plt.plot(Img3CR, Img3PSNR, '-o')
plt.plot(Img4CR, Img4PSNR, '-o')
plt.plot(Img651CR, Img651PSNR, '-o')
plt.legend(['image 1', 'image 2','image 3', 'image 4','image 651'])
# Add labels and title
plt.xlabel('CR')
plt.ylabel('PSNR')
plt.title('CR vs PSNR')

# Show the plot
plt.show()


In [ ]:
#Plot CR vs SSIM
plt.figure(figsize=(10,8))
# Create the plot
plt.plot(Img1CR, Img1ssim, '-o')
plt.plot(Img2CR, Img2ssim, '-o')
plt.plot(Img3CR, Img3ssim, '-o')
plt.plot(Img4CR, Img4ssim, '-o')
plt.plot(Img651CR, Img651ssim, '-o')
plt.legend(['image 1', 'image 2','image 3', 'image 4','image 651'])
# Add labels and title
plt.xlabel('CR')
plt.ylabel('SSIM')
plt.title('CR vs SSIM')

# Show the plot
plt.show()


In [ ]:
#CR vs Top-1/Top-5 Accuracy

plt.figure(figsize=(10,8))
# Create the plot

plt.plot(Img1CR,c1[0:10], '-o')


plt.plot(Img2CR,c1[0:10], '-o')

plt.plot(Img3CR,c1[0:10], '-o')

plt.plot(Img4CR,c1[0:10], '-o')

plt.plot(Img651CR,c1[0:10], '-o')
plt.legend(['Image 1 Inception','Image 2 Inception','Image 3 Inception','Image 4 Inception','Image 651 Inception'])
# Add labels and title
plt.xlabel('CR')
plt.ylabel('TOP1 accuracy %')
plt.title('CR vs Top-1')

# Show the plot
plt.show()

In [ ]:
#CR vs Top-5 Accuracy

plt.figure(figsize=(10,8))
# Create the plot
plt.plot(Img1CR,c5[0:10], '-o')


plt.plot(Img2CR,c5[0:10], '-o')

plt.plot(Img3CR,c5[0:10], '-o')

plt.plot(Img4CR,c5[0:10], '-o')

plt.plot(Img651CR,c5[0:10], '-o')

plt.legend(['Image 1 Inception','Image 2 Inception','Image 3 Inception','Image 4 Inception','Image 651 Inception'])
# Add labels and title
plt.xlabel('CR')
plt.ylabel('TOP5 accuracy %')
plt.title('CR vs Top-5')

# Show the plot
plt.show()

In [ ]:
from ctypes import c_uint16
#PSNR vs Top-1/Top-5 Accuracy

plt.figure(figsize=(10,8))
# Create the plot
plt.plot(Img1PSNR,c1[0:10], '-o')

plt.plot(Img2PSNR,c1[0:10], '-o')

plt.plot(Img3PSNR,c1[0:10], '-o')

plt.plot(Img4PSNR,c1[0:10], '-o')

plt.plot(Img651PSNR,c1[0:10], '-o')

# Add labels and title
plt.legend(['Image 1 Inception','Image 2 Inception','Image 3 Inception','Image 4 Inception','Image 651 Inception'])
plt.xlabel('PSNR')
plt.ylabel('TOP1 accuracy %')
plt.title('PSNR vs Top-1')

# Show the plot
plt.show()

In [ ]:
#PSNR vs Top-1/Top-5 Accuracy

plt.figure(figsize=(10,8))
# Create the plot

plt.plot(Img1PSNR,c5[0:10], '-o')

plt.plot(Img2PSNR,c5[0:10], '-o')


plt.plot(Img3PSNR,c5[0:10], '-o')


plt.plot(Img4PSNR,c5[0:10], '-o')


plt.plot(Img651PSNR,c5[0:10], '-o')

# Add labels and title
plt.legend(['Image 1 Inception','Image 2 Inception','Image 3 Inception','Image 4 Inception','Image 651 Inception'])
plt.xlabel('PSNR')
plt.ylabel('TOP5 accuracy %')
plt.title('PSNR vs Top-5')

# Show the plot
plt.show()